In [1]:
from IPython import display
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import PIL
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory
import os
import time

2023-06-01 09:36:35.981078: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
batch_size = 2
img_height = 184
img_width = 216
img_size = (img_height, img_width)
img_shape = img_size + (3,)

ds_path = "/drive0-storage/Gracia/dataset/hc"
# ds_path = "/drive0-storage/Gracia/dataset/bd"
# ds_path = "/drive0-storage/Gracia/dataset/sz"
train_ds = image_dataset_from_directory(ds_path,
                                        seed = 123,
                                        image_size = img_size,
                                        batch_size = batch_size)

Found 11895 files belonging to 1 classes.


2023-06-01 09:36:49.455740: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 09:36:49.904636: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1637] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7910 MB memory:  -> device: 0, name: A100-SXM4-40GB MIG 2g.10gb, pci bus id: 0000:90:00.0, compute capability: 8.0


In [3]:
class_names = train_ds.class_names
num_classes = len(class_names)

print(class_names)

['healthy_controls']


In [4]:
plt.figure(figsize=(7,5))
for images, labels in train_ds.take(1):
    for i in range(8):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.axis("off")

In [5]:
BUFFER_SIZE = 1000
AUTOTUNE = tf.data.AUTOTUNE

In [6]:
def preprocess(image, label):
    image = tf.image.rgb_to_grayscale(image)
    image = (image-127.5)/127.5
    return image

In [7]:
train_dataset = (
    train_ds
    .shuffle(buffer_size = BUFFER_SIZE)
    .map(preprocess, num_parallel_calls=AUTOTUNE)
    .prefetch(AUTOTUNE)
)

In [8]:
print(len(train_dataset))
for images in train_dataset.take(1):
    img = images[0]
    print(img.shape)  
    print(img.dtype)  
    print(np.min(img),np.max(img))  

1487
(184, 216, 1)
<dtype: 'float32'>
-1.0 0.8803172


In [9]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(23*27*512, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((23, 27, 512)))
    assert model.output_shape == (None, 23, 27, 512) 

    model.add(layers.Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 46, 54, 256)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 92, 108, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 184, 216, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (4, 4), strides=(1, 1), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 184, 216, 1)

    return model

In [10]:
generator = make_generator_model()
generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 317952)            31795200  
                                                                 
 batch_normalization (BatchN  (None, 317952)           1271808   
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 317952)            0         
                                                                 
 reshape (Reshape)           (None, 23, 27, 512)       0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 46, 54, 256)      3276800   
 nspose)                                                         
                                                                 
 batch_normalization_1 (Batc  (None, 46, 54, 256)      1

2023-06-01 09:35:26.005730: I tensorflow/stream_executor/cuda/cuda_blas.cc:1633] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-06-01 09:35:26.170304: I tensorflow/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600


In [11]:
noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)
plt.imshow(generated_image[0, :, :, 0], cmap='gray');

In [12]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[184, 216, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(512, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

In [13]:
discriminator = make_discriminator_model()
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 92, 108, 64)       1664      
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 92, 108, 64)       0         
                                                                 
 dropout (Dropout)           (None, 92, 108, 64)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 46, 54, 128)       204928    
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 46, 54, 128)       0         
                                                                 
 dropout_1 (Dropout)         (None, 46, 54, 128)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 23, 27, 256)      

In [14]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [15]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [16]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    noise = tf.random.normal([batch_size, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [17]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [18]:
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()

        for image_batch in dataset:
            train_step(image_batch)

        # Produce images for the GIF as you go
        display.clear_output(wait=True)
        generate_and_save_images(generator,
                                 epoch + 1,
                                 seed)

        # Save the model every 15 epochs
        if (epoch + 1) % 15 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)

        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

In [19]:
def generate_and_save_images(model, epoch, test_input):
    # Notice `training` is set to False.
    # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(3, 3))

    for i in range(predictions.shape[0]):
        plt.subplot(3, 3, i+1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')

    plt.savefig('Gracia/image_synthesis/hc/epoch_{:04d}.png'.format(epoch))
    plt.show()

In [18]:
EPOCHS = 1000
noise_dim = 100
num_examples_to_generate = 9
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
train(train_dataset, EPOCHS)

## Generate image

In [66]:
predictions = generator(seed, training=False)
# plt.imshow(predictions[8], cmap='gray');
test = predictions[8]
print(test.dtype)
print(np.min(test),np.max(test))
print(test.shape)

<dtype: 'float32'>
-1.0 0.8954076
(184, 216, 1)


orginal dataset

In [72]:
for images, labels in train_ds.take(1):
    for i in range(1):
        sample = images[i]
        print(sample.dtype)
        print(np.min(sample), np.max(sample))
        print(sample.shape)

<dtype: 'float32'>
0.0 244.78333
(184, 216, 3)


save generated images

In [88]:
import cv2
for i in range(3):
    img = np.array(predictions[i])
#     img = img.reshape(182, 218)
#     img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
    img = img/2 + 0.5
    img *= 255
    img = img.astype(np.uint8)
    cv2.imwrite(f"tes_hc{i}.png", img)

open image

In [91]:
im = PIL.Image.open(r"test/tes_hc0.png") 
# im = PIL.Image.open(r"dataset/hc/healthy_controls/11156_99.png") 
im = np.array(im)
print(im.dtype)
print(im.shape)
print(im.min(), im.max())

uint8
(184, 216)
0 241
